In [1]:
# Import relevant libraries
import re
import json
import copy
import requests 
import numpy as np
import pandas as pd
import sqlite3 as lite
import lxml.html as lh
from sqlite3 import Error
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq

In [2]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [3]:
# Step 1 : Compile a data containing all 34 pages of title, href

base_url = "https://www.abodo.com/ann-arbor-mi/university-of-michigan-ann-arbor-apartments/campus?page="
data_url = []

for page_number in range(1, 35):
    url = base_url+str(page_number)+"/"
    r = requests.get(url, timeout=(2, 10), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    soup = json.loads(soup.find('script', type='application/ld+json').text)
    for item in soup['itemListElement']:
        data_url.append(item['url'])

KeyboardInterrupt: 

In [4]:
all_property = pd.DataFrame(data_url, columns=['property_link'])
all_property.drop_duplicates()
all_property.to_csv('Abodo_Property_Overview_List.csv')

In [5]:
# Read Saved CSV Files

all_property      = pd.read_csv('Abodo_Property_Overview_List.csv')
amenities_dict_df = pd.read_csv('amenities_dict.csv')

amenities_dict_df.columns = ['features', 'val']
temp_df = pd.DataFrame({"features":['Room/Rental Price', 'Description', 'Additional Information'], "val":[' ', ' ', ' ']})
amenities_dict_df = amenities_dict_df.append(temp_df, ignore_index = True)
amenities_dict_df

,features,val
0,Heat Included,0
1,Air Conditioning,0
2,Furnished,0
3,Fitness Center,0
4,Trash Removal Included,0
5,Internet Included,0
6,Controlled Access,0
7,Water Included,0
8,High-speed Internet,0
9,24-hour Maintenance,0


In [6]:
part_1 = all_property[:200]
part_2 = all_property[200:400]
part_3 = all_property[400:]

In [7]:
# Step 2A : Create a list one time thing - inefficient, don't care (680)

amenities_list = []
count = 1
for link in part_1['property_link']:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        if item.text not in amenities_list:
            amenities_list.append(item.text)
    count += 1
    if (count%10) == 0:
        print(count)

for link in part_2['property_link']:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        if item.text not in amenities_list:
            amenities_list.append(item.text)
    count += 1
    if (count%10) == 0:
        print(count)
        
for link in part_3['property_link']:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        if item.text not in amenities_list:
            amenities_list.append(item.text)
    count += 1
    if (count%10) == 0:
        print(count)


# Step 2B : Create a Dictionary containing all variables and count    
amenities_dict = {}
for item in amenities_list:
    amenities_dict[item] = 0

# Step 2C : Convert to Dataframe and Save.
amenities_dict_df = pd.DataFrame.from_dict(amenities_dict, orient='index')
amenities_dict_df.to_csv('amenities_dict.csv')

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200


In [14]:
output1 = pd.DataFrame(0, index=part_1['property_link'], columns=amenities_dict_df.features, dtype=object)
print(output1.shape)
output2 = pd.DataFrame(0, index=part_2['property_link'], columns=amenities_dict_df.features, dtype=object)
print(output2.shape)
output3 = pd.DataFrame(0, index=part_3['property_link'], columns=amenities_dict_df.features, dtype=object)
print(output3.shape)

(200, 171)
(200, 171)
(280, 171)


In [15]:
# Step 3 : Output 
count = 1
for link in part_1['property_link']:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        output1[item.text][link] = 1
    if (soup.find_all('div', {'class': 'selected-floorplans-container'})):
        output1['Room/Rental Price'][link]       = soup.find_all('div', {'class': 'selected-floorplans-container'})
    if (soup.find_all('div', {'class': 'shortentext'})):
        output1['Description'][link]             = soup.find_all('div', {'class': 'shortentext'})
    if (soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})):
        output1['Additional Information'][link]  = soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})
    count += 1
    if (count%10) == 0:
        print(count)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200


In [44]:
output21 = pd.DataFrame(0, index=part_2['property_link'][:135], columns=amenities_dict_df.features, dtype=object)
output22 = pd.DataFrame(0, index=part_2['property_link'][136:], columns=amenities_dict_df.features, dtype=object)
print(output21.shape)
print(output22.shape)

(135, 171)
(64, 171)


In [45]:
count = 1
for link in part_2['property_link'][:135]:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        output21[item.text][link] = 1
    if (soup.find_all('div', {'class': 'selected-floorplans-container'})):
        output21['Room/Rental Price'][link]       = soup.find_all('div', {'class': 'selected-floorplans-container'})
    if (soup.find_all('div', {'class': 'shortentext'})):
        output21['Description'][link]             = soup.find_all('div', {'class': 'shortentext'})
    if (soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})):
        output21['Additional Information'][link]  = soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})
    count += 1
    if (count%10) == 0:
        print(count)

10
20
30
40
50
60
70
80
90
100
110
120
130


In [46]:
count = 1
for link in part_2['property_link'][136:]:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        output22[item.text][link] = 1
    if (soup.find_all('div', {'class': 'selected-floorplans-container'})):
        output22['Room/Rental Price'][link]       = soup.find_all('div', {'class': 'selected-floorplans-container'})
    if (soup.find_all('div', {'class': 'shortentext'})):
        output22['Description'][link]             = soup.find_all('div', {'class': 'shortentext'})
    if (soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})):
        output22['Additional Information'][link]  = soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})
    count += 1
    if (count%10) == 0:
        print(count)

10
20
30
40
50
60


In [30]:
count = 1
for link in part_3['property_link']:
    r = requests.get(link, timeout=(20, 200), headers=req_headers)
    c = r.content
    soup = BeautifulSoup(c,"lxml")
    amenities   = soup.find_all('span', {'class': 'amenity-name'})
    for item in amenities:
        output3[item.text][link] = 1
    if (soup.find_all('div', {'class': 'selected-floorplans-container'})):
        output3['Room/Rental Price'][link]       = soup.find_all('div', {'class': 'selected-floorplans-container'})
    if (soup.find_all('div', {'class': 'shortentext'})):
        output3['Description'][link]             = soup.find_all('div', {'class': 'shortentext'})
    if (soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})):
        output3['Additional Information'][link]  = soup.find_all('div', {'class': 'content-row-info col-xs-12 col-sm-8 no-pad'})
    count += 1
    if (count%10) == 0:
        print(count)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280


In [54]:
final_output = pd.concat([output1, output21, output22, output3])
print(final_output.shape)
final_output.to_excel('output.xlsx')

(679, 171)
